<a href="https://colab.research.google.com/github/RoyceAmando/SirkaMLdemo/blob/main/Sirka_Meal_Log.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import IPython
import time

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
PROJECT_ID = "studied-brand-426103-q3"
LOCATION = "us-central1"

import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
import requests
from google.colab import files
from IPython.display import display, Markdown
from vertexai.preview.generative_models import GenerativeModel, Image
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)

In [ ]:
model = GenerativeModel("gemini-1.5-pro-001")

In [ ]:
get_calorie_func = FunctionDeclaration(
    name= "food_analyze_result",
    description= "Get the analyze of given food",
    parameters= {
        "type": "object",
        "properties": {
            "food_name": {
                "type": "string",
                "description": "The name of the food from the prompt"
            },
            "calorie": {
                "type": "integer",
                "description": "Estimation of the calorie of the food in Kcal at the result"
            },
            "protein": {
                "type": "integer",
                "description": "Estimation of the protein of the food gram"
            },
            "carbohydrate": {
                "type": "integer",
                "description": "Estimation of the carbohydrate of the food in gram"
            },
            "fat": {
                "type": "integer",
                "description": "Estimation of the fat of the food in gram"
            },
            "vitamin_a": {
                "type": "string",
                "description": "Estimation of the vitamin_a of the food in category like (None,Low,Moderate,High)"
            },
            "vitamin_b": {
                    "type": "string",
                    "description": "Estimation of the vitamin_b of the food"
            },
            "vitamin_c": {
                    "type": "string",
                    "description": "Estimation of the vitamin_c of the food"
            },
            "vitamin_e": {
                    "type": "string",
                    "description": "Estimation of the vitamin_e of the food"
            },
            "calcium": {
                    "type": "string",
                    "description": "Estimation of the calcium of the food"
            },
            "iron": {
                    "type": "string",
                    "description": "Estimation of the iron of the food"
            },
            "pottasium": {
                    "type": "string",
                    "description": "Estimation of the pottasium of the food"
            },
            "sodium": {
                    "type": "string",
                    "description": "Estimation of the sodium of the food"
            },
        },
            "required": [
              "food_name",
              "calorie",
              "protein",
              "carbohydrate"
              "fat",
              "vitamin_a",
              "vitamin_b",
              "vitamin_c",
              "vitamin_e",
              "calcium",
              "iron",
              "pottasium",
              "sodium"
        ]
    }

)




In [ ]:
calorie_tool = Tool(
    function_declarations=[get_calorie_func],
)

In [ ]:
uploaded = files.upload()
IMAGE_FILE = next(iter(uploaded))
image = Image.load_from_file(IMAGE_FILE)

# Remove image_tool from the contents list
response = model.generate_content(["What is shown in this image?", image])

In [ ]:
import json

def extract_text_from_response(response: str) -> str:
    try:
        text = response.candidates[0].content.parts[0].text
        return text
    except (KeyError, IndexError, json.JSONDecodeError) as e:
        print(f"Error extracting text: {str(e)}")  # Debug print
        return f"Error extracting text: {str(e)}"


# Extract and print the text from the response
extracted_text = extract_text_from_response(response)
print(extracted_text)  # Debug print

In [ ]:
prompt = extracted_text

response = model.generate_content(
    prompt,
    tools=[calorie_tool],
)

In [ ]:
response.candidates[0].content.parts[0].function_call

In [ ]:
paramlist = {}
for key, value in response.candidates[0].content.parts[0].function_call.args.items():
    paramlist[key] = value
paramlist